<a href="https://colab.research.google.com/github/poketmonster/sentimentanalisys/blob/master/Entropia_ALC_SentimentAnalisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import csv, operator
import numpy as np
import nltk
import string
import re
import math
import random
import xml.etree.ElementTree as et
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from random import shuffle
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
!rm -r *
from google.colab import files

uploaded = files.upload() # then browse, select the files. It's then uploaded

Saving TASS2017_T1_development.xml to TASS2017_T1_development.xml
Saving TASS2017_T1_test.xml to TASS2017_T1_test.xml
Saving TASS2017_T1_training.xml to TASS2017_T1_training.xml


In [41]:
!rm -r *
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalisys/master/TASS2017_T1_development.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalisys/master/TASS2017_T1_test.xml
!wget https://raw.githubusercontent.com/poketmonster/sentimentanalisys/master/TASS2017_T1_training.xml

--2019-04-12 15:18:29--  https://raw.githubusercontent.com/poketmonster/sentimentanalisys/master/TASS2017_T1_development.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163475 (160K) [text/plain]
Saving to: ‘TASS2017_T1_development.xml’

TASS2017_T1_develop 100%[===================>] 159.64K  --.-KB/s    in 0.04s   

2019-04-12 15:18:29 (4.18 MB/s) - ‘TASS2017_T1_development.xml’ saved [163475/163475]

--2019-04-12 15:18:31--  https://raw.githubusercontent.com/poketmonster/sentimentanalisys/master/TASS2017_T1_test.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiti

In [0]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas DataFrame 
    with the given columns. The first element of df_cols is supposed to be 
    the identifier variable, which is an attribute of each node element in 
    the XML data; other features will be parsed from the text content of 
    each sub-element. """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    out_df = pd.DataFrame(columns = df_cols)
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index = True)
        
    return out_df

In [100]:
train = parse_XML("TASS2017_T1_training.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TRAIN")
train.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
print(train.head())

dev = parse_XML("TASS2017_T1_development.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("DEV")
dev.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
print(dev.head())

test = parse_XML("TASS2017_T1_test.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print("TEST")
test.columns = ["tweetid", "user", "content", "date", "lang", "polarity"]
print(test.head())

TRAIN
              tweetid           user  \
0  768213876278165504   OnceBukowski   
1  768213567418036224       anahorxn   
2  768212591105703936   martitarey13   
3  768221670255493120  endlessmilerr   
4  768221021300264964       JunoWTFL   

                                             content                 date  \
0  -Me caes muy bien \n-Tienes que jugar más part...  2016-08-23 22:30:35   
1  @myendlesshazza a. que puto mal escribo\n\nb. ...  2016-08-23 22:29:21   
2  @estherct209 jajajaja la tuya y la d mucha gen...  2016-08-23 22:25:29   
3  Quiero mogollón a @AlbaBenito99 pero sobretodo...  2016-08-23 23:01:33   
4  Vale he visto la tia bebiendose su regla y me ...  2016-08-23 22:58:58   

  lang polarity  
0   es     NONE  
1   es        N  
2   es        N  
3   es        P  
4   es        N  
DEV
              tweetid           user  \
0  770976639173951488   noseashetero   
1  771092421866389508       Templelx   
2  771092111429083136       esskuu94   
3  771092070572449

In [104]:

total = train
tknzr = TweetTokenizer()

positiveWords=()
negativeWords=()


otherstopwords=["http", "https", ",", ".", ";", ":", "@", "RT"]

size=len(total)
print(size)
train.at[2, 'tweetid']

1008


'768212591105703936'

In [128]:
for i in range(0,size):
    #print(total[i][1])
    senseurl = re.sub(r"http\S+", "", train.at[i, 'content'])
    tokenitzat = tknzr.tokenize(senseurl)
    #print(tokenitzat)
    if(train.at[i, 'polarity']=="P"): #Positivos
        print("POS "+train.at[i, 'content'])
        for word in tokenitzat:
            if word not in string.punctuation:
                if word not in stopwords.words('english') and word not in otherstopwords:
                    positiveWords+=(word.lower(), )
            
    elif(train.at[i, 'polarity']=="N"): #Negativos
        print("NEG "+train.at[i, 'content'])
        for word in tokenitzat:
            if word not in string.punctuation:
                if word not in stopwords.words('english') and word not in otherstopwords:
                    negativeWords+=(word.lower(), )

print("positive")
print(positiveWords)
print("negative")
print(negativeWords)

positiveWordfreq = {}
negativeWordfreq= {}

for word in positiveWords:
    count = positiveWordfreq.get(word,0)
    positiveWordfreq[word] = count + 1

for word in negativeWords:
    count = negativeWordfreq.get(word,0)
    negativeWordfreq[word] = count + 1


#print (str(negativeWordfreq))
#print (str(positiveWordfreq))

print(len(positiveWords))
print(len(negativeWords))

print(len(positiveWordfreq))
print(len(negativeWordfreq))

#Preparem training set

NEG @myendlesshazza a. que puto mal escribo

b. me sigo surrando help 

3. ha quedado raro el "cómetelo" ahí JAJAJAJA
NEG @estherct209 jajajaja la tuya y la d mucha gente seguro!! Pero yo no puedo sin mi melena me muero 
POS Quiero mogollón a @AlbaBenito99 pero sobretodo por lo rápido que contesta a los wasaps 
NEG Vale he visto la tia bebiendose su regla y me hs dado muchs grima 
POS @Yulian_Poe @guillermoterry1 Ah. mucho más por supuesto! solo que lo incluyo. Me habías entendido mal 
NEG Se ha terminado #Rio2016 Lamentablemente no arriendo las ganancias al pueblo brasileño por la penuria que les espera 
Suerte y solidaridad
POS 11. siiii fue super gracioso teniamos que habernos sacado una foto 
NEG @toNi_end seria mejor que dejasen de emitir esa basura ya  hay que evolucionar para bien y eso
NEG @jonoro96 te mandaria a comprarte un burro, pero no creo que hayan tiendas abiertas ahora 
NEG Mg y pongo un adjetivo super repelente a vuestro nombre 
NEG @hywzz la voz de María al final me 

In [0]:
def create_word_features(tweet, lang):
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    for word in tokenitzat:
            if word not in string.punctuation and word not in otherstopwords:
                if lang=="EN" and word not in stopwords.words('english'):
                    usables.append(word.lower())
                if lang=="ES" and word not in stopwords.words('spanish'):
                    usables.append(word.lower())
    
    diccionario=dict([word, True] for word in usables)
    #diccionario.update([lang, True])
    return diccionario
  
def create_word_featuresSTM(tweet, lang):
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    for word in tokenitzat:
            if word not in string.punctuation and word not in otherstopwords:
                if lang=="EN" and word not in stopwords.words('english'):
                    usables.append(word.lower())
                if lang=="ES" and word not in stopwords.words('spanish'):
                    usables.append(word.lower())
    
    stemmer = PorterStemmer()
    diccionario=dict([stemmer.stem(word), True] for word in usables)
    #diccionario.update([lang, True])
    return diccionario

def bigrams(tweet, lang):
    bigrams=[]
    i=0
    prev=False
    for word in tknzr.tokenize(tweet):
        if  word not in otherstopwords:
            if(not prev):
                prev=word
            else:
                bigrams.append(prev+" "+word)
                prev=word
            i+=1

    diccionario=dict([word, True] for word in bigrams)
    return diccionario

def trigrams(tweet, lang):
    trigrams=[]
    i=0
    word1=False
    word2=False
    for wordact in tknzr.tokenize(tweet):
        if  wordact not in otherstopwords:
            if(not word2):
                word2=wordact
            elif(not word1):
                word1=word2
                word2=wordact
            else:
                trigrams.append(word1+" "+word2+" "+wordact)
                word1=word2
                word2=wordact
            i+=1
            #print("Actual: "+str(word1)+" "+str(word2)+" "+str(wordact))

    diccionario=dict([word, True] for word in trigrams)
    return diccionario

def bigramaCaracter(tweet):
    charbigrams=[]
    bigrams = nltk.bigrams(tweet)
    for bi in bigrams:
        charbigrams.append(bi[0]+" "+bi[1])
    #print (str(charbigrams))

    diccionario=dict([word, True] for word in charbigrams)
    return diccionario
  
def posWordStm(tweet, lang):
    stemmer = PorterStemmer()
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    stemizat = [stemmer.stem(word) for word in tokenitzat]
    
    for word in stemizat:
      if word not in string.punctuation and word not in otherstopwords:
        if lang=="EN" and word not in stopwords.words('english'):
          usables.append(word.lower())
        if lang=="ES" and word not in stopwords.words('spanish'):
          usables.append(word.lower())
          
          
    pos= nltk.pos_tag(usables)

    diccionario=dict( word for word in pos)
    return diccionario
  
def posWord(tweet, lang):
    stemmer = PorterStemmer()
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    
    for word in tokenitzat:
      if word not in string.punctuation and word not in otherstopwords:
        if lang=="EN" and word not in stopwords.words('english'):
          usables.append(word.lower())
        if lang=="ES" and word not in stopwords.words('spanish'):
          usables.append(word.lower())
    
    
    pos= nltk.pos_tag(usables)

    diccionario=dict( word for word in pos)
    return diccionario
  

def  features(tweet, lang):
    #Traiem fora les URL
    tweet=re.sub(r"http\S+", "", tweet)
    #Eliminem repeticions de la mateixa lletra més de 2 cops
    tweet=re.sub(r'(.)\1+', r'\1\1', tweet) 
    #Eliminem referències a usuaris
    tweet=re.sub(r"@\S+", "", tweet)
    
    words=create_word_featuresSTM(tweet, lang)
    #words=create_word_features(tweet, lang)
    #words=posWordStm(tweet, lang)
    #words=posWord(tweet, lang)
    #Twbigrams=bigrams(tweet, lang)
    #Twtrigrams=trigrams(tweet, lang)
    #charbigrams=bigramaCaracter(tweet)

    diccionario = words.copy()
    #diccionario = Twbigrams.copy()
    #diccionario.update(Twbigrams)
    #diccionario.update(Twtrigrams)
    #diccionario.update(charbigrams)

    return diccionario

In [129]:
for i in range(200,300):
    print (train.at[i,'content'])
    print(features(train.at[i,'content'], train.at[i,'lang'].upper()))


Me veis en persona, no me saludais y luego me hablais para decirme que me habeis visto 
{'me': True, 'vei': True, 'persona': True, 'saludai': True, 'luego': True, 'hablai': True, 'decirm': True, 'habei': True, 'visto': True}
@BlasAuryn si es que no puedo olvidar cuando te viii, imposible. Que resumiendo, no te puedo olvidar  y no lo haré jamás chiqui ❣️💞
{'si': True, 'puedo': True, 'olvidar': True, 'vii': True, 'impos': True, 'que': True, 'resumiendo': True, 'haré': True, 'jamá': True, 'chiqui': True, '❣': True, '️': True, '💞': True}
@DTM_Spain    En las carreras es al revé a más interés, más coches  ... pero tú mismo lo dices, es una empresa, aunque se venda ...
{'en': True, 'carrera': True, 'revé': True, 'interé': True, 'coch': True, '..': True, 'mismo': True, 'dice': True, 'empresa': True, 'aunqu': True, 'venda': True}
@Garcaterra estoy total y absolutamente desesperanzada  @ahorapodemos
{'total': True, 'absolutament': True, 'desesperanzada': True}
@Sorayita_push Bien bien, si estan

In [132]:
negative_tweets = []
positive_tweets = []

for i in range(0,size):
    if(train.at[i,'polarity']=="N"):
        negative_tweets.append((features(train.at[i,'content'] , train.at[i,'lang'].upper()),0))
    if(train.at[i,'polarity']=="P"):
        positive_tweets.append((features(train.at[i,'content'] , train.at[i,'lang'].upper()),0))
       

train_set=positive_tweets+negative_tweets
print ("long total")
print (len(train_set))


maxent_classifier = nltk.classify.MaxentClassifier.train(train_set,max_iter=25)

long total
736
  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1           0.00000        1.000
             2           0.00000        1.000
             3           0.00000        1.000
             4           0.00000        1.000
             5           0.00000        1.000
             6           0.00000        1.000
             7           0.00000        1.000
             8           0.00000        1.000
             9           0.00000        1.000
            10           0.00000        1.000
            11           0.00000        1.000
            12           0.00000        1.000
            13           0.00000        1.000
            14           0.00000        1.000
            15           0.00000        1.000
            16           0.00000        1.000
            17           0.00000        1.000
            18           0.00000        1.000
            19           0.00000 

In [122]:
for i in range(100,150):
    print (dev.at[i,'content'])
    print(features(dev.at[i,'content'], dev.at[i,'lang'].upper()))


negative_tweets_dev = []
positive_tweets_dev  = []

for i in range(0,size):
    if(dev.at[i,'polarity']=="N"):
        negative_tweets_dev.append((features(dev.at[i,'content'] , dev.at[i,'lang'].upper()),0))
    if(dev.at[i,'polarity']=="P"):
        positive_tweets_dev.append((features(dev.at[i,'content'] , dev.at[i,'lang'].upper()),0))
       

dev_set=positive_tweets_dev+negative_tweets_dev


print ("long total")
print (len(dev_set))


@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito 
{'aún': True, 'leído': True, 'caerán': True, 'prontito': True}
@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito 
{'aún': True, 'leído': True, 'caerán': True, 'prontito': True}
@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito 
{'aún': True, 'leído': True, 'caerán': True, 'prontito': True}
@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito 
{'aún': True, 'leído': True, 'caerán': True, 'prontito': True}
long total
1008


In [121]:
for tweet in dev_set:
    #print(tweet[1])
    test_class = maxent_classifier.classify(tweet[0])
    class_prob = maxent_classifier.prob_classify(tweet[0])
    #print("calculada: "+str(test_class)+"  --  real: "+str(tweet[1]))
    #if str(test_class)!=str(tweet[1]):
        #for label in class_prob.samples():
            #print("%s: %f" % (label, class_prob.prob(label)))
   

accuracy = nltk.classify.util.accuracy(maxent_classifier, dev_set)

print("Accuracy: ")
print(accuracy*100)


print("Features: ")
print(maxent_classifier.show_most_informative_features(100))


Accuracy: 
100.0
Features: 
   0.000 puto==True and label is 0
   0.000 mal==True and label is 0
   0.000 escribo==True and label is 0
   0.000 b==True and label is 0
   0.000 sigo==True and label is 0
   0.000 surrando==True and label is 0
   0.000 help==True and label is 0
   0.000 3==True and label is 0
   0.000 quedado==True and label is 0
   0.000 raro==True and label is 0
   0.000 cómetelo==True and label is 0
   0.000 ahí==True and label is 0
   0.000 jajajaja==True and label is 0
None
